# 第 10 讲：递归与面向对象编程 - 学习笔记

本讲涵盖了两个核心主题：
1.  **递归 (Recursion)**：一种强大的解决问题的技巧，其核心是一个函数调用其自身。
2.  **面向对象编程 (Object-Oriented Programming, OOP)**：一种通过将数据和行为捆绑到“对象”中来组织程序的编程范式。

---

## 第一部分：字典 (复习)

*(幻灯片 2-15 页是对上一讲词频统计例子的回顾。核心要点是：字典非常适合用于计数或将唯一的键映射到对应的值。)*

我们快速回顾并重构一下那个例子的最终代码。

In [ ]:
# 歌曲文本
song = "RAH RAH AH AH AH ROM MAH RO MAH MAH"

# 函数：创建词频字典
def generate_word_dict(song_text):
    word_dict = {}
    words_list = song_text.lower().split()
    for word in words_list:
        if word in word_dict:
            word_dict[word] += 1
        else:
            word_dict[word] = 1
    return word_dict

# 函数：找到出现频率最高的单词
def find_frequent_word(word_dict):
    if not word_dict:
        return ([], 0)
    highest_freq = max(word_dict.values())
    words = [word for word, freq in word_dict.items() if freq == highest_freq]
    return (words, highest_freq)

# 函数：找到所有出现频率高于阈值 'x' 的单词
def occurs_often(original_word_dict, x):
    word_dict = original_word_dict.copy()
    result_list = []
    while True:
        words, freq = find_frequent_word(word_dict)
        if freq > x:
            result_list.append((words, freq))
            for word in words:
                del word_dict[word]
        else:
            break
    return result_list

# --- 运行代码 ---
word_frequencies = generate_word_dict(song)
print(f"词频字典: {word_frequencies}")

most_common = find_frequent_word(word_frequencies)
print(f"出现最多次的单词: {most_common}")

frequent_list = occurs_often(word_frequencies, 1)
print(f"出现次数 > 1 的单词列表: {frequent_list}")

---

## 第二部分：递归 (Recursion)

到目前为止，我们一直使用 **迭代**（`for` 和 `while` 循环）来解决需要重复执行的问题。递归是另一种可选的强大方法。

**核心思想**：一个递归函数通过将一个大问题分解成一个更小、更简单的、但**问题性质完全相同**的子问题来解决它。然后，它调用**自身**来解决这个更小的子问题。

每一个递归函数都必须包含两个关键部分：

1.  **基本情况 (Base Case)**：这是问题的最简化版本，可以被直接解决，无需再次进行递归调用。这是递归的**终止条件**，防止函数无限地调用自己。
2.  **递归步骤 (Recursive Step)**：函数在这一部分会调用自身，但输入的参数会有所改变，使其更接近“基本情况”。它利用子问题的解来构建出原问题的解。

### 例子 1：用递归实现乘法

让我们思考一下 `a * b`。我们可以用加法来定义它：
`5 * 4 = 5 + 5 + 5 + 5`

请注意这个模式：
- `5 * 4 = 5 + (5 * 3)`
- `5 * 3 = 5 + (5 * 2)`
- `5 * 2 = 5 + (5 * 1)`

在这里，`mult(a, b)` 的问题被简化为了 `a + mult(a, b-1)`。

- **递归步骤**: `mult(a, b) = a + mult(a, b-1)`
- **基本情况**: 最简单的情况是什么？是当 `b` 等于 1 的时候。`mult(a, 1)` 的结果就是 `a`。

In [ ]:
# 使用迭代实现乘法 (用于对比)
def mult_iter(a, b):
    total = 0
    for _ in range(b):
        total += a
    return total

# 使用递归实现乘法
def mult_recur(a, b):
    # 基本情况：如果 b 是 1，结果就是 a
    if b == 1:
        return a
    # 递归步骤：解决一个更小的问题 (a * (b-1))，然后将 'a' 加到结果上
    else:
        return a + mult_recur(a, b - 1)

print(f"迭代法 5 * 4 = {mult_iter(5, 4)}")
print(f"递归法 5 * 4 = {mult_recur(5, 4)}")

### 递归的可视化：调用栈 (Call Stack)

计算机是如何跟踪所有这些函数调用的？它使用了一个叫做“调用栈”的概念。想象一叠盘子，每当一个函数被调用时，一个新的“盘子”（代表该函数独立的运行环境和变量）就被放到最上面。一个函数只有在它上面那个盘子所代表的函数返回结果后，才能执行完毕。

让我们来追踪一下幻灯片中的 `fact(4)`（阶乘）的执行过程：

1.  `fact(4)` 被调用。它无法直接求解，需要 `fact(3)` 的结果。它准备返回 `4 * fact(3)`。
2.  `fact(3)` 被调用。它需要 `fact(2)` 的结果。它准备返回 `3 * fact(2)`。
3.  `fact(2)` 被调用。它需要 `fact(1)` 的结果。它准备返回 `2 * fact(1)`。
4.  `fact(1)` 被调用。**这就是基本情况！** 它直接返回 `1`。
5.  现在，调用栈开始“退栈”：
    -   `fact(2)` 接收到返回值 `1`，计算 `2 * 1`，然后返回 `2`。
    -   `fact(3)` 接收到返回值 `2`，计算 `3 * 2`，然后返回 `6`。
    -   `fact(4)` 接收到返回值 `6`，计算 `4 * 6`，最终返回 `24`。

每一次函数调用都是完全独立的，拥有自己私有的变量 `n` 的副本。

In [ ]:
# 幻灯片中的阶乘函数
def fact(n):
    # 基本情况: 1 的阶乘是 1
    if n == 1:
        return 1
    # 递归步骤: n! = n * (n-1)!
    else:
        return n * fact(n - 1)

result = fact(4)
print(f"fact(4) = {result}")

### “你来试试！”练习

#### 1. 递归的幂函数
完成一个函数来计算 n^p。
- 递归关系: `n^p = n * n^(p-1)`
- 基本情况: `n^0 = 1`

In [ ]:
def power_recur(n, p):
  # 基本情况：任何数的 0 次方都是 1
  if p == 0:
    return 1
  # 递归步骤
  else:
    return n * power_recur(n, p - 1)

print(f"2^3 = {power_recur(2, 3)}") # 期望输出: 8
print(f"5^4 = {power_recur(5, 4)}") # 期望输出: 625

### 何时使用递归？

当一个问题本身就具有**天然的递归结构**时，递归方法会大放异彩。

**例子**：文件系统。一个文件夹可以包含文件和**其他文件夹**。要在一个文件夹里查找文件，你先检查当前层级。如果遇到了一个子文件夹，你需要在那个子文件夹里执行**完全相同的查找过程**。

另一个经典例子是处理嵌套列表，即列表中的元素本身也可能是列表。

#### 2. 深度反转一个嵌套列表

In [ ]:
# 这个函数会递归地反转一个列表，包括它内部的所有子列表
def deep_rev(L):
    # 创建一个新列表来存放反转后的元素
    result_list = []
    
    # 以相反的顺序遍历原始列表
    for element in L[::-1]:
        # 检查当前元素是否是列表
        if isinstance(element, list):
            # 如果是列表，就进行递归调用来反转这个子列表
            result_list.append(deep_rev(element))
        else:
            # 如果不是列表，就直接添加
            result_list.append(element)
            
    return result_list

nested_list = [[1, 2], 3, 4, [[5, 6], [7, 8]]]
reversed_list = deep_rev(nested_list)

print(f"原始列表: {nested_list}")
print(f"深度反转后的列表: {reversed_list}")
# 期望输出: [[[[8, 7], [6, 5]], 4, 3, [2, 1]]]

#### 3. 统计嵌套列表中的整数个数

In [ ]:
def count_elem(L):
    """
    返回一个嵌套整数列表中所有整数的总数。
    """
    count = 0
    for element in L:
        if isinstance(element, list):
            # 如果元素是列表，就将递归调用的结果累加到计数器上
            count += count_elem(element)
        else:
            # 如果是整数，计数器加一
            count += 1
    return count

print(f"列表 [1, 2, [[3, 4], 5]] 中的整数总数: {count_elem([1, 2, [[3, 4], 5]])}") # 期望输出: 5

---

## 第三部分：使用 Class（类）进行面向对象编程 (OOP)

OOP 是一种模仿现实世界来组织代码的方式。在现实世界里，我们身边充满了各种“对象”（比如一辆车、一个人、一只猫），它们都拥有：
1.  **状态 (数据)**：也叫属性。比如，一辆车有它的 `颜色`、`当前速度`、`油量`。
2.  **行为 (方法)**：它能执行的动作。比如，一辆车可以 `启动引擎()`、`加速()`、`刹车()`。

在 Python 中，**万物皆对象**。一个列表（list）就是一个对象，它有自己的数据（列表中的元素）和方法（`.append()`、`.sort()` 等）。

OOP 允许我们使用 `class` 关键字来定义我们自己的、全新的对象类型。一个 **类 (class)** 就像一个 **蓝图** 或模板。而一个 **实例 (instance)** 则是根据这个蓝图创造出来的真实的对象。

### 定义一个类：`Coordinate`（坐标）的例子

让我们来为二维坐标点创建一个蓝图。

**设计决策**：
- **数据**：一个坐标点需要什么？一个 `x` 值和一个 `y` 值。
- **行为**：我们能对坐标点做什么？我们可以计算两个坐标点之间的 `距离 (distance)`。

In [ ]:
import math

class Coordinate(object):
    # 1. 初始化方法 / 构造方法 (`__init__`)
    # 当一个新实例被创建时，这个特殊方法会被自动调用。
    # 它的任务是设置对象的初始状态（即数据属性）。
    def __init__(self, xval, yval):
        # `self` 指代正在被创建的那个具体的实例。
        # 我们正在创建数据属性 `x` 和 `y`，并将它们“绑定”到这个实例上。
        self.x = xval
        self.y = yval

    # 2. 自定义方法
    # 方法就是定义在类内部的函数。
    # 它们总是把 `self` 作为第一个参数，这个参数由 Python 自动传入。
    # `self` 让方法能够访问到实例自己的数据。
    def distance(self, other_coordinate):
        # self.x 是调用这个方法的实例的 x 值。
        # other_coordinate.x 是作为参数传入的另一个 Coordinate 对象的 x 值。
        x_diff_sq = (self.x - other_coordinate.x)**2
        y_diff_sq = (self.y - other_coordinate.y)**2
        return math.sqrt(x_diff_sq + y_diff_sq)
    
    # 定义一个 __str__ 方法是一个好习惯，它能控制对象如何被打印。
    def __str__(self):
        return f"坐标({self.x}, {self.y})"

# --- 使用我们定义的类 ---

# 1. 从蓝图创建实例（对象）
# 这会调用 __init__ 方法。`c` 会成为 `self`，3 会传递给 `xval`，4 会传递给 `yval`。
c = Coordinate(3, 4) 
origin = Coordinate(0, 0)

# 2. 使用点（.）标记法访问数据属性
print(f"点 c 的坐标是 x={c.x}, y={c.y}")
print(f"原点的坐标是 x={origin.x}, y={origin.y}")

# 3. 使用点（.）标记法调用方法
# 当你调用 `c.distance(origin)` 时，Python 实际上在后台做了这件事：
# 调用 `distance` 方法，并将 `c` 作为 `self` 参数传入，
# 将 `origin` 作为 `other_coordinate` 参数传入。
dist = c.distance(origin)

print(f"对象 c 是: {c}") # 这里会调用 __str__ 方法
print(f"点 {c} 和原点 {origin} 之间的距离是 {dist:.2f}")

### OOP 核心概念总结

-   **类 (Class)**：对象的蓝图或模板 (例如 `Coordinate`)。
-   **实例/对象 (Instance/Object)**：根据蓝图创造出来的具体事物 (例如 `c`, `origin`)。每个实例在内存中都是一个独立的实体。
-   **`self`**：类的方法中的一个特殊变量，它总是指向调用该方法的那个具体实例。这是对象访问其**自身**数据的方式。
-   **`__init__`**：特殊的构造方法，用于初始化一个实例的初始属性。
-   **属性 (Attribute)**：与对象关联的一段数据 (例如 `c.x`)。
-   **方法 (Method)**：与对象关联的一个函数，定义了它的行为 (例如 `c.distance()`)。
-   **封装 (Encapsulation)**：将相关的数据（属性）和行为（方法）捆绑在一个包（即类）中的思想。这能让你的代码更有条理、更模块化。

> dive into CLASS

# Python Class 深入拓展学习笔记

欢迎来到 `class` 的世界！我们已经知道，类（Class）是创建对象（Object）的蓝图。现在，让我们跳出幻灯片的基础知识，探索一些能让你真正发挥面向对象编程（OOP）威力的概念。

## 1. 深入理解 `self`：它到底是什么？

幻灯片中提到，方法（method）的第一个参数总是 `self`。这是新手最容易困惑的地方之一。

**核心要点**：`self` **不是** Python 的关键字，它只是一个被广泛遵守的**约定名称**。它的作用是**代表类的实例本身**。当你调用一个实例的方法时，Python 会自动将这个实例作为第一个参数传递进去。

让我们用代码来揭开它的“魔法”面纱：

In [ ]:
class Cat:
    def __init__(self, name):
        # self.name 将 'name' 绑定到这个特定的实例上
        self.name = name

    def speak(self):
        # self.name 访问这个实例自己的 name 属性
        print(f"{self.name} says: Meow!")

# 创建一个 Cat 的实例
my_cat = Cat("Mochi")

# --- 这是我们通常的调用方式 ---
# Python 在背后偷偷地做了转换：my_cat.speak() -> Cat.speak(my_cat)
my_cat.speak()

# --- 这揭示了 self 的本质 ---
# 我们可以直接通过类来调用方法，但必须手动传入一个实例作为第一个参数
another_cat = Cat("Luna")
Cat.speak(another_cat) # 这里，`another_cat` 就是 `speak` 方法中的 `self`

## 2. 特殊方法 (Special Methods / Dunder Methods)

幻灯片展示了 `__init__`，这是众多特殊方法中的一个。这些以双下划线开头和结尾的方法（我们称之为 "Dunder Methods"，来自 Double Underscore）能让你的自定义对象拥有和 Python 内置类型一样的行为。

它们是 Python OOP 的精髓所在，能让你的代码变得极其优雅和直观。

### `__str__` 和 `__repr__`：控制对象的打印方式

-   `__str__`： 当你使用 `print()` 函数或 `str()` 转型时被调用。目标是**易于人类阅读**。
-   `__repr__`：当你直接在解释器中输入变量名，或者使用 `repr()` 函数时被调用。目标是**无歧义**，最好能让开发者看到这个字符串就知道如何重建这个对象。

让我们来给 `Coordinate` 类添加这两个方法。

In [ ]:
import math

class Coordinate:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    # 用于 print()，更友好
    def __str__(self):
        return f"一个位于 ({self.x}, {self.y}) 的坐标点"
        
    # 用于开发者调试，更精确
    def __repr__(self):
        return f"Coordinate(x={self.x}, y={self.y})"

# 创建实例
point = Coordinate(3, 4)

# 现在看看打印效果的区别
print(point)            # 调用 __str__
print(str(point))       # 显式调用 __str__
print(repr(point))      # 显式调用 __repr__

# 在 Jupyter Notebook 或 Python 解释器中，直接输入变量会调用 __repr__
point

### `__eq__`：定义对象间的相等性 (`==`)

默认情况下，两个实例即使所有属性都相同，它们用 `==` 比较的结果也是 `False`，因为它们是内存中不同的对象。我们可以通过定义 `__eq__` 方法来改变这个行为。

In [ ]:
class Coordinate:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __repr__(self):
        return f"Coordinate({self.x}, {self.y})"

    # 定义两个 Coordinate 实例何时被视为“相等”
    def __eq__(self, other):
        # 首先检查对方是不是也是 Coordinate 类型
        if isinstance(other, Coordinate):
            # 如果 x 和 y 属性都相等，那它们就相等
            return self.x == other.x and self.y == other.y
        return False

p1 = Coordinate(1, 2)
p2 = Coordinate(1, 2)
p3 = Coordinate(3, 4)
p4 = [1, 2] # 这是一个列表，不是 Coordinate 对象

print(f"p1 和 p2 是同一个对象吗? {p1 is p2}")
print(f"p1 和 p2 的值相等吗? {p1 == p2}") # 如果没有 __eq__，这里会是 False
print(f"p1 和 p3 的值相等吗? {p1 == p3}")
print(f"p1 和 p4 的值相等吗? {p1 == p4}")

### `__add__`：让对象支持 `+` 运算 (运算符重载)

想让两个坐标点能够相加吗？定义 `__add__` 方法即可！这被称为**运算符重载**。

In [ ]:
class Coordinate:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __repr__(self):
        return f"Coordinate({self.x}, {self.y})"
    
    # 定义 '+' 运算符的行为
    def __add__(self, other):
        if isinstance(other, Coordinate):
            # 返回一个新的 Coordinate 实例，其 x 和 y 是两者之和
            new_x = self.x + other.x
            new_y = self.y + other.y
            return Coordinate(new_x, new_y)
        # 如果加号右边的不是 Coordinate，就抛出错误
        raise TypeError("Unsupported operand type for +: 'Coordinate' and '{type(other).__name__}'")

p1 = Coordinate(1, 2)
p2 = Coordinate(10, 20)
result = p1 + p2 # 这看起来就像内置的数字加法一样自然！

print(f"{p1} + {p2} = {result}")

try:
    p1 + 5
except TypeError as e:
    print(f"尝试与整数相加时出错: {e}")

## 3. 实例属性 vs. 类属性

这是一个非常重要的概念区分。

-   **实例属性 (Instance Attributes)**：属于**单个实例**的属性。它们通常在 `__init__` 方法中通过 `self.attribute = value` 来定义。每个实例都可以有不同的值。(`c.x`, `origin.x`)
-   **类属性 (Class Attributes)**：属于**整个类**的属性。它被这个类的**所有实例共享**。类属性直接在 `class` 代码块下定义，不带 `self`。

类属性非常适合用来存储所有实例都应共享的数据，比如常量或者一个用来追踪实例数量的计数器。

In [ ]:
class Robot:
    # 这是一个类属性，被所有 Robot 实例共享
    population = 0
    
    def __init__(self, name):
        # 这是实例属性，每个 robot 都有自己的名字
        self.name = name
        print(f"机器人 {self.name} 已被制造！")
        
        # 通过 类名.属性名 来增加共享的计数器
        Robot.population += 1

    def destroy(self):
        print(f"机器人 {self.name} 已被摧毁！")
        Robot.population -= 1

    # 这个方法也操作类属性
    def say_hi(self):
        print(f"你好！我的名字是 {self.name}。")
    
    # 这是一个类方法，它只与类有关，与具体实例无关
    @classmethod
    def how_many(cls):
        # 在类方法中，我们用 `cls` 代替 `self` 来指代类本身
        print(f"我们现在有 {cls.population} 个机器人。")

# --- 使用 Robot 类 ---
print(f"初始机器人数量: {Robot.population}")
Robot.how_many()

r1 = Robot("R2-D2")
r2 = Robot("C-3PO")

print("\n--- 创建了两个机器人后 ---")
# 我们可以通过类或任何一个实例来访问类属性
print(f"通过类访问: {Robot.population}")
print(f"通过实例 r1 访问: {r1.population}") # 不推荐这样做，但可以
r1.how_many() # 也可以通过实例调用类方法

print("\n--- 摧毁一个机器人 ---")
r2.destroy()
Robot.how_many()

## 4. 继承 (Inheritance)：构建类的层级关系

继承是 OOP 的三大支柱之一（另外两个是封装和多态）。它允许我们创建一个新类，这个新类可以**继承**一个已存在的父类（也叫基类）的所有属性和方法。

这极大地促进了代码的**重用**。子类可以添加新功能，也可以**重写 (override)** 父类的方法来实现自己的特定行为。

让我们创建一个 `ThreeDCoordinate` 类，它继承自我们已经写好的 `Coordinate` 类。

In [ ]:
# 这是我们的父类
class Coordinate:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def distance_from_origin(self):
        return math.sqrt(self.x**2 + self.y**2)

    def __repr__(self):
        return f"Coordinate({self.x}, {self.y})"

# 这是子类，它继承了 Coordinate 的所有东西
class ThreeDCoordinate(Coordinate):
    # 1. 重写 __init__ 方法
    def __init__(self, x, y, z):
        # `super()` 是一个特殊函数，可以调用父类的方法
        # 这里我们调用父类的 __init__ 来处理 x 和 y
        super().__init__(x, y)
        # 然后我们只处理子类新增的属性
        self.z = z

    # 2. 重写 distance_from_origin 方法以适应 3D 空间
    def distance_from_origin(self):
        d_2d_sq = self.x**2 + self.y**2 # 我们可以直接访问继承来的 self.x 和 self.y
        return math.sqrt(d_2d_sq + self.z**2)
    
    # 3. 重写 __repr__ 以显示更完整的信息
    def __repr__(self):
        return f"ThreeDCoordinate({self.x}, {self.y}, {self.z})"


# --- 使用子类 ---
p2d = Coordinate(3, 4)
p3d = ThreeDCoordinate(3, 4, 12)

print(f"2D 点: {p2d}, 离原点距离: {p2d.distance_from_origin():.2f}")
print(f"3D 点: {p3d}, 离原点距离: {p3d.distance_from_origin():.2f}")

## 5. 注意事项与最佳实践

1.  **命名约定**：
    -   类名使用 `PascalCase`（或 `CapWords`）风格，即每个单词首字母大写，例如 `MyClassName`。
    -   方法名和属性名使用 `snake_case` 风格，即全小写并用下划线分隔，例如 `my_method_name`。

2.  **保持 `__init__` 简洁**：`__init__` 方法的主要职责是接收参数并设置实例的初始属性。避免在里面放入复杂的逻辑。

3.  **警惕可变的默认参数**：这是一个 Python 中非常经典的“陷阱”。**永远不要**在函数或方法的定义中使用列表或字典作为默认参数。

    ```python
    # 错误的示范
    class BadExample:
        def add_item(self, item, item_list=[]):
            item_list.append(item)
            return item_list

    # 让我们看看会发生什么
    b1 = BadExample()
    b2 = BadExample()
    print(b1.add_item(1)) # 输出
    print(b1.add_item(2)) # 输出
    # b2 应该有一个全新的列表，但是...
    print(b2.add_item(3)) # 惊了！输出
    # 因为默认列表 `[]` 只在函数定义时创建一次，所有调用都共享它！
    ```
    **正确的做法**是使用 `None`作为默认值，然后在方法内部创建新列表。
    ```python
    # 正确的示范
    class GoodExample:
        def add_item(self, item, item_list=None):
            if item_list is None:
                item_list = []
            item_list.append(item)
            return item_list

    g1 = GoodExample()
    g2 = GoodExample()
    print(g1.add_item(1)) # 输出
    print(g1.add_item(2)) # 输出
    print(g2.add_item(3)) # 输出，现在它们是独立的了！
    ```
4.  **编写文档字符串 (Docstrings)**：为你的类和方法编写清晰的文档字符串，解释它们是做什么的、接受什么参数、返回什么。这对于团队协作和未来的你都至关重要。

---

希望这份拓展笔记能帮助你更深入地理解和运用 Python 的 `class`。面向对象编程是一个宏大的主题，但掌握了这些核心概念后，你就有了构建任何复杂系统的基础。大胆去实验和创造吧！